# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [1]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

Não há linhas duplicadas, pois após a checagem das linhas e colunas e posteriormente, a execução do método drop_duplicates , 
o número de linhas e colunas permaneceu o mesmo.

In [3]:
#verificando as primeiras linhas do DataFrame
sinasc.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


In [5]:
#verificando as colunas presentes no DataFrame
sinasc.columns

Index(['ORIGEM', 'CODESTAB', 'CODMUNNASC', 'LOCNASC', 'IDADEMAE', 'ESTCIVMAE',
       'ESCMAE', 'CODOCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'CODMUNRES',
       'GESTACAO', 'GRAVIDEZ', 'PARTO', 'CONSULTAS', 'DTNASC', 'HORANASC',
       'SEXO', 'APGAR1', 'APGAR5', 'RACACOR', 'PESO', 'IDANOMAL', 'DTCADASTRO',
       'CODANOMAL', 'NUMEROLOTE', 'VERSAOSIST', 'DTRECEBIM', 'DIFDATA',
       'DTRECORIGA', 'NATURALMAE', 'CODMUNNATU', 'CODUFNATU', 'ESCMAE2010',
       'SERIESCMAE', 'DTNASCMAE', 'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR',
       'QTDPARTCES', 'IDADEPAI', 'DTULTMENST', 'SEMAGESTAC', 'TPMETESTIM',
       'CONSPRENAT', 'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO',
       'TPNASCASSI', 'TPFUNCRESP', 'TPDOCRESP', 'DTDECLARAC', 'ESCMAEAGR1',
       'STDNEPIDEM', 'STDNNOVA', 'CODPAISRES', 'TPROBSON', 'PARIDADE',
       'KOTELCHUCK', 'CONTADOR', 'munResStatus', 'munResTipo', 'munResNome',
       'munResUf', 'munResLat', 'munResLon', 'munResAlt', 'munResArea'],
      dtype='object')

In [7]:
sinasc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27028 entries, 0 to 27027
Data columns (total 69 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ORIGEM        27028 non-null  int64  
 1   CODESTAB      26913 non-null  float64
 2   CODMUNNASC    27028 non-null  int64  
 3   LOCNASC       27028 non-null  int64  
 4   IDADEMAE      27028 non-null  int64  
 5   ESTCIVMAE     26711 non-null  float64
 6   ESCMAE        26716 non-null  object 
 7   CODOCUPMAE    24121 non-null  float64
 8   QTDFILVIVO    25455 non-null  float64
 9   QTDFILMORT    24930 non-null  float64
 10  CODMUNRES     27028 non-null  int64  
 11  GESTACAO      25796 non-null  object 
 12  GRAVIDEZ      26949 non-null  object 
 13  PARTO         26979 non-null  object 
 14  CONSULTAS     27028 non-null  int64  
 15  DTNASC        27028 non-null  object 
 16  HORANASC      27007 non-null  float64
 17  SEXO          27024 non-null  object 
 18  APGAR1        26932 non-nu

In [9]:
# 2) verificando os dados ausentes por variável:
sinasc.isna().any()


ORIGEM        False
CODESTAB       True
CODMUNNASC    False
LOCNASC       False
IDADEMAE      False
              ...  
munResUf      False
munResLat      True
munResLon      True
munResAlt      True
munResArea     True
Length: 69, dtype: bool

Algumas colunas apresentam dados ausentes.

In [11]:
#Verificando a quantidade de dados ausentes por coluna
sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [13]:
# 3) Recontando as colunas com valores faltanres que possuem algum interesse
sinasc.loc[:,['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',   'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']].isna().sum()


LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [15]:
# 4) Removendo os dados faltantes na coluna 'APGAR5'
sinasc = sinasc.dropna(subset=['APGAR5'])

In [9]:
#Verificando dados ausentes na coluna de interesse
sinasc['APGAR5'].isna().sum()

0

In [17]:
#verificando a nova quantidade de linhas e colunas
sinasc.shape

(26925, 69)

In [19]:
# 5) Substituindo valores não preenchidos pelo código 9:
sinasc['ESTCIVMAE'].fillna(9,inplace=True)


In [21]:
# Verificando a nova proporção de dados ausentes
sinasc['ESTCIVMAE'].isna().sum()

0

In [23]:
#Verificando as linhas e colunas do dataframe
sinasc.shape

(26925, 69)

In [25]:
# 6) Substituindo valores de QTDFILVIVO or 0
sinasc['QTDFILVIVO'].fillna(0,inplace=True)
sinasc.shape

(26925, 69)

In [27]:
# 7) Preenchendo os demais dados do dataframe
sinasc.loc[:,['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',   'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']].isna().sum()



LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [35]:
#verificando valores ignorados na sessão de CONSULTAS
sinasc['CONSULTAS'].value_counts()

CONSULTAS
4    18587
3     5957
2     1359
1      972
9       50
Name: count, dtype: int64

In [37]:
#Calculando a média de consultas para preenchimento de dados ignorados com a média.
sinasc['CONSULTAS'].mean()

3.5787929433611887

In [39]:
#Foi atribuído valores 9 (ignorado), pois nas colunas de 'escolaridade', 'tempo de gestação' e 'tipo de gravidez' 
# temos variáveis de intervalo e do tipo str, inviabilizando o preenchimento por média ou mediana. Foi atribuído uma média = a 3
# para a coluna de consultas
sinasc['CONSULTAS'].fillna(3,inplace=True)
sinasc['ESCMAE'].fillna(9,inplace=True)
sinasc['GESTACAO'].fillna(9,inplace=True)
sinasc['GRAVIDEZ'].fillna(9,inplace=True)

In [41]:
sinasc.loc[:,['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',   'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']].isna().sum()


LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64

In [43]:
# 8) Classificação de axfixia de acordo com os parâmetros
sinasc.loc[sinasc['APGAR5'] >= 8, 'cass_asfix'] = 'normal'
sinasc.loc[(sinasc['APGAR5'] >= 6 )& (sinasc['APGAR5'] < 8),
           'cass_asfix'] = 'asfixia leve'
sinasc.loc[(sinasc['APGAR5'] >=4) & (sinasc['APGAR5'] < 6),
           'cass_asfix'] = 'asfixia moderada'
sinasc.loc[sinasc['APGAR5'] < 4, 'cass_asfix'] = 'asfixia severa'

In [45]:
#determinação da frequência
sinasc['cass_asfix'].value_counts()

cass_asfix
normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: count, dtype: int64

In [47]:
# 9) Renomeando as variáveis de acordo com sake case
sinasc.rename(columns={'ORIGEM':'origem', 'CODESTAB':'code_estab', 'CODMUNNASC':'code_mun_nasc', 'LOCNASC':'loc_nasc',
                       'IDADEMAE':'idade_mae', 'ESTCIVMAE':'estado_civ_mae','ESCMAE':'esc_mae', 'CODOCUPMAE':'cod_ocup_mae',
                       'QTDFILVIVO':'qtd_fil_vivo', 'QTDFILMORT':'qtd_fil_mort', 'CODMUNRES':'cod_mun_res','GESTACAO':'gestacao', 
                       'GRAVIDEZ':'gravidez', 'PARTO':'parto', 'CONSULTAS':'consultas', 'DTNASC':'data_nasc', 'HORANASC':'hora_nasc',
                       'SEXO':'sexo', 'APGAR1':'apgar1', 'APGAR5':'apgar5', 'RACACOR':'raca_cor', 'PESO':'peso', 'IDANOMAL':'id_anomalia',
                       'DTCADASTRO':'dt_cadastro','CODANOMAL':'cod_anomalia', 'NUMEROLOTE':'numero_lote', 'VERSAOSIST':'versao_sist',
                       'DTRECEBIM':'dt_recebim', 'DIFDATA':'dif_data','DTRECORIGA':'dt_rec_origa', 'NATURALMAE':'natural_mae', 'CODMUNNATU':'cod_mun_natu',
                       'CODUFNATU':'cod_uf_natu', 'ESCMAE2010':'esc_mae_2010','SERIESCMAE':'serie_esc_mae', 'DTNASCMAE':'dt_nasc_mae', 
                       'RACACORMAE':'raca_cor_mae', 'QTDGESTANT':'qtd_gest_ant', 'QTDPARTNOR':'qtd_part_nor','QTDPARTCES':'qtd_part_ces', 
                       'IDADEPAI':'idade_pai', 'DTULTMENST':'dt_ult_menst', 'SEMAGESTAC':'sema_gestac', 'TPMETESTIM':'tpmetestim',
                       'CONSPRENAT':'cons_prenat', 'MESPRENAT':'mes_prenat', 'TPAPRESENT':'tp_apresent', 'STTRABPART':'st_trab_part', 
                       'STCESPARTO':'st_ces_parto','TPNASCASSI':'tp_nasc_assi', 'TPFUNCRESP':'tp_func_resp', 'TPDOCRESP':'tp_doc_resp', 
                       'DTDECLARAC':'dt_declarac', 'ESCMAEAGR1':'esc_mae_agr1','STDNEPIDEM':'stdn_epidem', 'STDNNOVA':'stdn_nova', 'CODPAISRES':'cod_pais_res', 
                       'TPROBSON':'tp_robson', 'PARIDADE':'par_idade','KOTELCHUCK':'kotel_chuck', 'CONTADOR':'contador', 'munResStatus':'mun_res_status', 
                       'munResTipo':'mun_res_tipo', 'munResNome':'mun_res_nome','munResUf':'mun_res_uf', 'munResLat':'mun_res_lat',
                       'munResLon':'mun_res_lon', 'munResAlt':'mun_res_alt', 'munResArea':'mun_res_area'
                      }, 
              inplace=True)


In [49]:
#verificando as colunas
sinasc.columns

Index(['origem', 'code_estab', 'code_mun_nasc', 'loc_nasc', 'idade_mae',
       'estado_civ_mae', 'esc_mae', 'cod_ocup_mae', 'qtd_fil_vivo',
       'qtd_fil_mort', 'cod_mun_res', 'gestacao', 'gravidez', 'parto',
       'consultas', 'data_nasc', 'hora_nasc', 'sexo', 'apgar1', 'apgar5',
       'raca_cor', 'peso', 'id_anomalia', 'dt_cadastro', 'cod_anomalia',
       'numero_lote', 'versao_sist', 'dt_recebim', 'dif_data', 'dt_rec_origa',
       'natural_mae', 'cod_mun_natu', 'cod_uf_natu', 'esc_mae_2010',
       'serie_esc_mae', 'dt_nasc_mae', 'raca_cor_mae', 'qtd_gest_ant',
       'qtd_part_nor', 'qtd_part_ces', 'idade_pai', 'dt_ult_menst',
       'sema_gestac', 'tpmetestim', 'cons_prenat', 'mes_prenat', 'tp_apresent',
       'st_trab_part', 'st_ces_parto', 'tp_nasc_assi', 'tp_func_resp',
       'tp_doc_resp', 'dt_declarac', 'esc_mae_agr1', 'stdn_epidem',
       'stdn_nova', 'cod_pais_res', 'tp_robson', 'par_idade', 'kotel_chuck',
       'contador', 'mun_res_status', 'mun_res_tipo', 'mun_

In [51]:
sinasc.head()

,origem,code_estab,code_mun_nasc,loc_nasc,idade_mae,estado_civ_mae,esc_mae,cod_ocup_mae,qtd_fil_vivo,qtd_fil_mort,...,contador,mun_res_status,mun_res_tipo,mun_res_nome,mun_res_uf,mun_res_lat,mun_res_lon,mun_res_alt,mun_res_area,cass_asfix
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,normal
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,normal
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,normal
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273,normal
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025,normal
